In [5]:
import os
import sys
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import gensim
import types


In [6]:
class Text_Net(nn.Module):

    def __init__(self):
        super(Text_Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=8, out_channels=52, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=52, out_channels=52, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=52, out_channels=52, kernel_size=3)
        self.fc1 = nn.Linear(in_features=52*73, out_features=9216)

    def forward(self, inputs):
        out = F.relu(self.conv1(out))
        out = F.max_pool1d(F.relu(self.conv2(out)), kernel_size=2)
        out = F.max_pool1d(F.relu(self.conv3(out)), kernel_size=2)
        out = self.fc1(out.view(-1, 52*73))
        return out

In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F
# class Text_Net(nn.Module):
#     def __init__(self, vocab_size, embed_dim, num_class):
#         super().__init__()
#         self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
#         self.fc = nn.Linear(embed_dim, num_class)
#         self.init_weights()

#     def init_weights(self):
#         initrange = 0.5
#         self.embedding.weight.data.uniform_(-initrange, initrange)
#         self.fc.weight.data.uniform_(-initrange, initrange)
#         self.fc.bias.data.zero_()

#     def forward(self, text, offsets):
#         embedded = self.embedding(text, offsets)
#         return self.fc(embedded)

In [8]:
# Load Google's pre-trained Word2Vec model.
#model = gensim.models.KeyedVectors.load_word2vec_format('words/GoogleNews-vectors-negative300.bin', binary=True)  

In [9]:
class Text_Net2(nn.Module):
    def __init__(self, dense, channels, kernels, paddings, keep_probab,
                 vocab_size, embedding_length, weights):
        super(CNN, self).__init__()

        self.dense = dense
        self.channels = channels
        self.kernels = kernels
        self.paddings = paddings
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length

        self.word_embeddings = self.__create_embedding(weights)
        self.conv1 = self.__create_convolution(0)
        self.conv2 = self.__create_convolution(1)
        self.conv3 = self.__create_convolution(2)
        self.dropout = nn.Dropout(keep_probab)
        self.result = self.__create_dense()

    def __create_dense(self):
        return nn.Sequential(
            nn.Linear(self.dense[0], self.dense[1]),
            nn.ReLU(),

            nn.Linear(self.dense[1], self.dense[2]),
            nn.ReLU(),

            nn.Linear(self.dense[2], self.dense[3]),
            nn.ReLU(),

            nn.Linear(self.dense[3], self.dense[4]),
            nn.ReLU(),

            nn.Softmax()
        )

    def __create_embedding(self, weights):
        word_embeddings = nn.Embedding(self.vocab_size, self.embedding_length)
        word_embeddings.weight = nn.Parameter(weights, requires_grad=False)
        return word_embeddings

    def __create_convolution(self, step):
        return nn.Sequential(
            nn.Conv2d(
                in_channels=self.channels[step],
                out_channels=self.channels[step+1],
                kernel_size=(self.kernels[step], self.embedding_length if step == 0 else 1),
                padding=(self.paddings[step], 0),
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(self.kernels[step], 1), padding=(1, 0), stride=(1, 1)),
        )

    def forward(self, x):

        x = self.word_embeddings(x)
        max_out1 = self.conv1(x)
        max_out2 = self.conv2(max_out1)
        max_out3 = self.conv3(max_out2)

        # all_out = torch.cat((max_out1, max_out2, max_out3), 0)
        dropout = self.dropout(max_out3)
        logits = self.result(dropout.flatten())

        return logits